This notebook applies **Appendix A Transformation of probability density** of *Craske J. et al. 2024* to compute the time evolution of joint probability density $f_\mathbf{Y}$ of the vertical velocity $Y^1_t$ and buoyancy field $Y^2_t$ 

\begin{align}
  Y_{1} &=\varphi_{t}^{1}(X_{1},X_{2}):=\frac{\sqrt{2}}{\pi}\left(k^{2}+\pi^{2}\right)a_{1}(t)\mathrm{cos}(kX_{1})\mathrm{sin}(\pi X_{2}), \\
  Y_{2} &=\varphi_{t}^{2}(X_{1},X_{2}):=\frac{\sqrt{2}}{\pi r}a_{2}(t)\mathrm{cos}(kX_{1})\mathrm{sin}(\pi X_{2})-\frac{1}{\pi r}a_{3}(t)\mathrm{sin}(2\pi X_{2}),
\end{align}

on a horizontally periodic domain $\mathcal{X} := [0,2\pi/k)\times [0,1]\ni \mathbf{X}:=(X_{1},X_{2})$, which evolve according to the Lorenz (1963) model 

\begin{align}
  \frac{da_{1}}{dt} &= s(a_{2}-a_{1}),\\
  \frac{da_{2}}{dt} &= ra_{1}-a_{2}-a_{1}a_{3},\\
  \frac{da_{3}}{dt} &= a_{1}a_{2}-ba_{3},
\end{align}
where $b:=4\pi^{2}(k^{2}+\pi^{2})^{-1}$ characterises the aspect ratio of the domain, $s$ is the Prandtl number and $r$ is a renormalised Rayleigh number.

We first define the fields and this systems of equations below along with helper functions to define the coefficients

In [ ]:
import numpy as np

def calc_coeffs(r=28,b=8/3):
    k = np.sqrt( (4-b)*np.pi**2/b )
    c0 = np.sqrt(2)/np.pi*(k**2+np.pi**2)
    c1 = np.sqrt(2)/np.pi/r
    c2 = 2/r/np.pi
    return [c0, c1, c2]

def wavenumber(b=8/3):
    """wavenumber k for a given parameter b"""
    return np.sqrt( (4-b)*np.pi**2/b )

def field(X, a, s=10,r=28,b=8/3):
    """Return the vertical velocity and buoyancy field for a given state a(t)"""
    k = wavenumber(b)
    c = calc_coeffs(r=r,b=b)

    # Vertical velocity
    Y0 = c[0]*a[0]*np.cos(k*X[0])*np.sin(np.pi*X[1])

    # Buoyancy field
    Y1 = c[1]*a[1]*np.cos(k*X[0])*np.sin(np.pi*X[1])-c[2]*a[2]*np.sin(2*np.pi*X[1])/2
    return [Y0, Y1]

def tangent(t, a, s=10, r=28, b=8/3):
    """Phase space velocity of the Lorenz equations."""
    da = [0]*3
    da[0] = s * (a[1]-a[0])
    da[1] = a[0] * (r - a[2]) - a[1]
    da[2] = a[0] * a[1] - b * a[2]
    return da

Note that the default parameters for all functions are set to $s = 10, r = 28, b = 8/3$ but these can be modified by the user. To generate the coefficients $a = [a_1(t), a_2(t), a_3(t)]$ as a function of time we simulate the Lorenz system using scipy.

In [ ]:
from scipy.integrate import solve_ivp

nt = 5000
tend = 200
dt = tend/nt
t = np.linspace(0,tend,nt+1)
sol = solve_ivp(tangent, np.array([0,tend]), np.array([1.0,1.0,8.0]), t_eval=t)
a = sol.y.T